# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [68]:
#Your code here

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from statsmodels.stats.proportion import proportions_ztest
%matplotlib inline

df=pd.read_csv('homepage_actions.csv')

df.head()
df['action'].unique()

array(['view', 'click'], dtype=object)

In [18]:
df.action.unique()

array(['view', 'click'], dtype=object)

In [19]:
df.describe()

,id
count,8188.000000
mean,564699.749878
std,219085.845672
min,182988.000000
25%,373637.500000
50%,566840.500000
75%,758078.000000
max,937217.000000


In [31]:
df.isnull().sum()

timestamp    0
id           0
group        0
action       0
dtype: int64

In [27]:
pivot_table = pd.pivot_table(df, index='action', columns='group', values='id', aggfunc='count')
print(pivot_table)

group   control  experiment
action                     
click       932         928
view       3332        2996


,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view
5,2016-09-24 20:32:25.712659,681598,experiment,view
6,2016-09-24 20:39:03.248853,522116,experiment,view
7,2016-09-24 20:57:20.336757,349125,experiment,view
8,2016-09-24 20:58:01.948663,349125,experiment,click
9,2016-09-24 21:00:12.278374,560027,control,view


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

Since clicks and views are dependent on one another, we should conduct a proportions z-test. I will compare the click through rate of the experimental and controls groups. My p_value is 0.05.

### First, we will calculate CTR for the control group.

In [42]:
#Control group Views
control_views=len(df[(df['group']=='control')&(df['action']=='view')])

In [43]:
#Control group clicks
control_clicks=len(df[(df['group']=='control')&(df['action']=='click')])

In [56]:
#Control CTR
control_ctr=control_clicks/control_views
print(control_CTR)

0.2797118847539016


### Second, we will calculate CTR for the experimental group.

In [39]:
#Experimental group views
experiment_views=len(df[(df['group']=='experiment')&(df['action']=='view')])

In [40]:
#Experimental group clicks
experiment_clicks=len(df[(df['group']=='experiment')&(df['action']=='click')])

In [55]:
#Experiment CTR
experiment_ctr=experiment_clicks/experiment_views
print(experiment_CTR)

0.3097463284379172


### Now we will conduct the proportions z test

In [50]:
count = np.array([control_clicks, experiment_clicks])
nobs = np.array([control_views, experiment_views])
stat, pval = proportions_ztest(count, nobs)


In [51]:
print(f"Z-statistic: {stat}")
print(f"P-value: {pval}")


Z-statistic: -2.618563885349469
P-value: 0.008830075576595804


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [58]:
# Expected number of clicks for the experiment group, assuming the same CTR as the control group
experiment_expected_clicks = experiment_views * control_ctr
print(f"Expected clicks for the experiment group: {experiment_expected_clicks}")

Expected clicks for the experiment group: 838.0168067226891


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [64]:
#Your code here
variance = experiment_views * control_ctr * (1 - control_ctr)
std_dev = np.sqrt(variance)
z_score = (experiment_clicks - experiment_expected_clicks) / std_dev
print(f"Z-score: {z_score}")

Z-score: 3.6625360854823588


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [70]:
#Your code here
p_value = 2 * (1 - norm.cdf(np.abs(z_score)))
print(f"P-value: {p_value}")


P-value: 0.0002497305601389943


### Analysis:

Does this result roughly match that of the previous statistical test?


The p values are different for each but they both have a very low p vlaue which leads you to reject the null hypothesis so the website should implement whatever they are showing the experimental group to increase CTR.

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.